# 数据加载、存储和文件格式

输入输出通常可以划分为几个大类:读取文本文件和其他更高效的磁盘存储格式,加载数据库中的数据,利用Web API操作网络资源；

In [45]:
import pandas as pd
from pandas import Series, DataFrame
import numpy as np

import sys
import csv,json
#from lxml.html import parse
import lxml.html
#from urllib.request import urlopen

%matplotlib inline

ImportError: libicui18n.so.58: cannot open shared object file: No such file or directory

## 读写文本格式的数据

因为其简单的文件交互语法、直观的数据结构,以及诸如元组打包解包之类的便利功能,Python在文本和文件处理方面已经成为一门招人喜欢的语言；

pandas提供了一些用于将表格型数据读取为DataFrame对象的函数，以下是总结,其中read_csv和read_table可能会是你今后用得最多的：
* read_csv：从文件、URL、文件型对象中加载带分隔符的数据，默认分隔符为逗号（,）；
* read_table：从文件、URL、文件型对象中加载带分隔符的数据，默认分隔符为制表符（\t）；
* read_fwf：读取定宽列格式数据（也就是说没有分隔符）；
* read_clipboard：读取剪切板中的数据，可以看到是read_table的剪切板版，在将网页转换成表格时很有用；

文本数据转换为DataFrame：
* 索引：将一个或者多个列当做返回的DataFrame处理，以及是否从文件、用户获取列名；
* 类型推断和数据转换：包括用户定义值的转换、缺失值标记列表等；
* 日期解析：包括组合功能，比如将分散在多个列中的时间信息组合成单个列；
* 迭代：支持对大文件进行逐块迭代；
* 不规整数据问题：跳过一些行、注释、页脚或其他一些不重要的东西（比如由成千上万个逗号隔开的数值数据）；

### 类型推断 -- 不需要手动指定数据的类型

In [2]:
pd.read_csv('data/ex1.csv')

,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


In [3]:
pd.read_csv('data/ex2.csv', header=None) # 显示说明数据中没有表头，同时会默认使用默认标签

,0,1,2,3,4
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


In [4]:
pd.read_csv('data/ex2.csv', names=['one','two','three','en']) # 显示指定表头

,one,two,three,en
1,2,3,4,hello
5,6,7,8,world
9,10,11,12,foo


In [5]:
# 将en列转换为行索引，书里那种通过index_col的方式会报错keyerror，不懂为啥，是不是必须要原有的标签才能设置啊

pd.read_csv('data/ex2.csv', names=['one','two','three','en']).set_index('en')

,one,two,three
en,,,
hello,2,3,4
world,6,7,8
foo,10,11,12


In [6]:
pd.read_csv('data/ex3.csv', index_col=['key1','key2']) # 层次化索引指定index_col使用序列即可

value1  value2
key1 key2                
one  a          1       2
     b          3       4
     c          5       6
     d          7       8
two  a          9      10
     b         11      12
     c         13      14
     d         15      16

In [7]:
pd.read_csv('data/ex4.csv', sep='\s+') # 指定分隔符，且使用正则语法实现多个空格的匹配

,a,b,c
aaa,123,-123,-234
bbb,-222,123,333
ccc,111,-567,-324


**注意**：上述read_csv方法读取后默认将aaa，bbb，ccc作为索引，这是因为第一行（pandas认为的表头）只有三个元素，而后续每行都有四个，因此pandas认为后续每行的第一列是索引，而不是数据值；

In [8]:
pd.read_csv('data/ex5.csv', skiprows=[0,2,4]) # 使用skeprows参数跳过不需要处理的行

,a,b,c,d,message
0,1,2,3,4,hello
1,9,10,11,12,foo


In [9]:
!cat data/ex6.csv

# 不同列会有不同的值对应到pandas中的NaN
# a:null
# b:NULL
# c:NA
# d:nil
# e:empty
# 同时，默认什么都没有也是NaN
a,b,c,d,e
1,,2,3,empty
null,2,NA,nil,5
3,NULL,4,nil,empty
null,4,NA,5,
5,6,7,8,9


In [10]:
pd.read_csv('data/ex6.csv', skiprows=range(7), na_values={
    'a':'null','b':'NULL','c':'NA','d':'nil','e':'empty'
}) # 指定每一列在源数据中映射到pandas中NaN的可能性，可以一列指定多个哦

,a,b,c,d,e
0,1.0,NaN,2.0,3.0,NaN
1,NaN,2.0,NaN,NaN,5.0
2,3.0,NaN,4.0,NaN,NaN
3,NaN,4.0,NaN,5.0,NaN
4,5.0,6.0,7.0,8.0,9.0


#### read_csv/read_table参数

* path：文件系统位置、URL、文件型对象；
* sep/delimiter：指定分隔符，支持正则；
* header：用作列名的行号，默认为0，如果没有列名可以指定为None；
* index_col：用作行索引的列，可以是单个列名/编号或者多个列名/编号；
* names：显示指定列名，通常在header=None之后使用；
* skiprows：需要忽略的行数，或者是需要跳过的行号序列集合；
* na_values：一组用于替换成NaN的值；
* comment：用于将注释信息从行尾拆分出去的字符（一个或多个）；
* parse_dates：尝试将数据解析为日期，默认是False，如果为True则解析所有列，还可以指定需要解析的一组列名或列号，如果列表的元素为列表或者元组，就会将多个列组合到一起进行解析（比如日期时间分开两列的数据）；
* keep_date_col：如果连接多列解析日期，则保持参与连接的列，默认是False；
* converters：由列名/列号跟函数之间的映射关系组成的字典，例如{'foo':f}，会对foo列的所有元素应用函数f；
* dayfirst：解析日期有歧义时，比如7/6/2012，将其看做国际格式，也就是2012年6月7号，默认是False；
* date_parser：用于解析日期的函数；
* nrows：需要读取的行数（从文件开始处算起）；
* iterator：返回一个TextParser以便逐块读取文件；
* chunksize：文件块大小；
* skip_footer：需要忽略的行数（从文件末尾算起）；
* verbose：打印各种解析器输出信息，比如“非数值列中缺失值的数量”；
* encoding：指定编码格式；
* squeeze：如果数据解析后只有一列，则返回Series；
* thousands：千分位分隔符，入“,”和“.”；

### 读取部分文本文件

#### 只想读取几行 -- nrows

In [11]:
pd.read_csv('data/ex7.csv', nrows=5)

,a,b,c,d
0,1,2,3,4
1,1,2,3,4
2,1,2,3,4
3,1,2,3,4
4,1,2,3,4


#### 逐块读取 -- chunksize

In [12]:
pd.read_csv('data/ex7.csv', chunksize=10) # 得到的是一个TextFileReader

In [13]:
chunker = pd.read_csv('data/ex7.csv', chunksize=1000)
tot = Series([])
for piece in chunker: # 遍历文件，按照chunksize指定的大小
    tot = tot.add(piece['d'].value_counts(), fill_value=0)
tot.sort_values(ascending=False)

8    41728.0
2    41728.0
4    28672.0
dtype: float64

### 将文本写出到文本格式

In [14]:
df = DataFrame({'CH':[66,77,88],'EN':[44,55,33],'MT':[22,88,99]}, index=['HL','NM','LL'])
df.ix[1:,1] = np.nan
df

/home/helong/anaconda2/envs/python2/lib/python2.7/site-packages/ipykernel_launcher.py:2: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  


,CH,EN,MT
HL,66,44.0,22
NM,77,NaN,88
LL,88,NaN,99


In [15]:
# 将DataFrame写出到csv文件，指定特殊分隔符，指定NaN值，不需要索引，指定需要的列和输出顺序
df.to_csv(sys.stdout, sep='>', na_rep='nil', index=False, columns=['MT','EN'])

MT>EN
22>44.0
88>nil
99>nil


### 手工处理分隔符格式

有些情况下，文件中的数据格式上有一些问题，那么直接使用read_table读取是会出错的，比如下面这个文件：

In [16]:
!cat data/ex8.csv

"a","b","c"
"1","2","3"
"1","2","3","4"


直接调用 pd.read_csv('data/ex8.csv') 报错ParserError；

这个时候就需要手工处理一下文件，使其符合pandas的读取格式要求了（所谓手工指的是手工写代码去处理，并不是手动去改哈）；

In [28]:
lines = list(csv.reader(open('data/ex8.csv')))
header = lines[0]
values = lines[1:]
data = {h:v for h,v in zip(header, zip(*values))}
print data

{'a': ('1', '1'), 'c': ('3', '3'), 'b': ('2', '2')}


In [30]:
DataFrame(data) # 经过手工处理后的dict符合DataFrame格式要求

,a,b,c
0,1,2,3
1,1,2,3


#### csv.reader常用参数

* delimiter：用于分割字段的单字符字符串，默认是逗号；
* lineterminator：用于写操作的行结束符，默认为'\r\n'，读操作将忽略此项，它能认出跨平台的行结束符；
* quotechar：用于带有特殊字符“如分隔符”的字段的引用符号，默认为“"”；
* quoting：引用约定；
* skipinitialspace：忽略分隔符后面的空白符，默认为False；
* doublequote：如何处理字段内的引用符号，如果为True，则双写；
* escapechar：用于对分隔符进行转义的字符串，默认禁用；

**PS**：如果分隔符较复杂，且含多个字符，那么就无法使用csv模块来处理，这种情况通常只能使用split或者正则来处理；

### JSON数据 -- json模块

In [34]:
json_str = """
{"name": "Wes",
"places_lived": ["United States", "Spain", "Germany"],
"pet": null,
"siblings": [{"name": "Scott", "age": 25, "pet": "Zuko"},
{"name": "Katie", "age": 33, "pet": "Cisco"}]
}
"""
json_obj = json.loads(json_str) # json.dumps是该函数的逆函数
print json_obj

{u'pet': None, u'siblings': [{u'pet': u'Zuko', u'age': 25, u'name': u'Scott'}, {u'pet': u'Cisco', u'age': 33, u'name': u'Katie'}], u'name': u'Wes', u'places_lived': [u'United States', u'Spain', u'Germany']}


In [35]:
DataFrame(json_obj['siblings']) # 通过json_obj中的一个子元素来构建DataFrame对象

,age,name,pet
0,25,Scott,Zuko
1,33,Katie,Cisco


### XML和HTML：Web数据 -- lxml.html，lxml.objectify

#### 解析HTML

#### 解析XML

## 二进制格式数据

### python内置的pickle格式

In [46]:
df = pd.read_csv('data/ex1.csv')
df

,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


In [50]:
df.to_pickle('data/test_pickle') # 等价于pd.to_pickle(df, 'data/test_pickle')
pd.read_pickle('data/test_pickle') # pd.read_pickle读取一个pickle序列化，生成DataFrame对象

,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


**警告**: pickle仅建议用于短期存储格式，其原因是很难保证该格式永远是稳定的，**今天pickle的对象可能无法被后续版本的库unpickle**出来；虽然我尽力保证这种事情不会发生在pandas中，但是今后的某个时候说不定还是得“打破”该pickle格式；

**警告2**：pd.to_pickle替换pd.save，pd.read_pickle替换pd.load

### HDF5格式 -- pandas.HDFStore 处理海量数量（无法放入内存时，需要本地化）

一个流行的工业级库,它是一个C库,带有许多语言的接口,如Java、Python和MATLAB等。HDF5中的HDF指的是层次型数据格式(hierarchical data format)。每个HDF5文件都含有一个**文件系统式的节点结构**,它使你能够**存储多个数据集并支持元数据**。与其他简单格式相比,HDF5支持多种压缩器的**即时压缩**,还能更高效地存储重复模式数据。对于那些非常大的无法直接放入内存的数据集,HDF5就是不错的选择,因为它可以**高效地分块读写**。

Python中的HDF5库有两个接口(即PyTables和h5py),它们各自采取了不同的问题解决方式。**h5py**提供了一种**直接而高级的HDF5API访问接口**,而**PyTables**则抽象了HDF5的许多细节以提供多种**灵活的数据容器、表索引、查询功能以及对核外计算技术**(out-of-corecomputation)的某些支持。

pandas有一个最小化的类似于字典的**HDFStore**类,它**通过PyTables存储pandas对象**。

如果需要处理**海量数据**,我建议你好好研究一下PyTables和h5py,看看它们能满足你的哪些需求。由于许多数据分析问题**都是IO密集型**(而**不是CPU密集型***),利用HDF5这样的工具能显著提升应用程序的效率。

**警告**: HDF5不是数据库。它最适合用作“一次写多次读”的数据集。虽然数据可以在任何时候被添加到文件中,但如果同时发生多个写操作,文件就可能会**被破坏**。

### Microsoft Excel格式 -- pandas.ExcelFile

In [58]:
# xls = pd.ExcelFile('data/excel.xls').parse() 与下述等价
xls = pd.read_excel('data/excel.xls', sheetname=0) # 默认就是0，如果指定为1，则会加载第二个sheet
xls

,Case,Persons age,Cheese type,Log
0,Old guy,42,stilton,Old man stilton
1,Young guy,21,cheddar,Young man cheddar


#### ExcelFile.parse/read_excel对比

* ExcelFile.parse：
    * 加载sheet代码复杂；
    * 加载同一个excel文件的多个sheet时，只需要加载一次文件，效率更高；
* read_excel：
    * 加载sheet代码简单；
    * 加载同一个excel文件时需要多次load文件本身，比较损失性能；
    
**结论**：如果只是加载一个excel文件的一个sheet，那么使用read_excel，如果加载多个且文件较大，那么使用ExcelFile更合理；

## 使用HTML的web api

许多网站都有一些通过JSON或其他格式提供数据的公共API。通过Python访问这些API的办法有不少。一个简单易用的办法(推荐)是requests包(http://docs.python-requests.org)。

## 使用数据库

许多应用中,数据很少取自文本文件,因为用这种方式**存储大量数据很低效**。基于SQL的关系型数据库(如SQL Server、PostgreSQL和MySQL等)使用非常广泛,此外还有一些非SQL(即所谓的NoSQL)型数据库也变得非常流行。数据库的选择通常取决于**性能**、**数据完整性**以及**应用程序的伸缩性**需求。

### SQL数据库

pandas有一个可以简化该过程的read_frame函数(位于pandas.io.sql模块)，只需传入select语句和连接对象即可：
```
import pandas.io.sql as sql
con = sqlite3.connect(':memory:')
sql.read_frame('select * from test', con)
```

### NoSQL数据库 -- 例如MongoDB

NoSQL数据库有许多不同的形式。有些是简单的**字典式键值对存储**(如BerkeleyDB和Tokyo Cabinet),另一些则是**基于文档**的(其中的基本单元是字典型的对象)。存储在MongoDB中的文档被**组织在数据库的集合**(collection)中。MongoDB服务器的每个运行实例可以有**多个**数据库,而每个数据库又可以有**多个**集合。